<a href="https://colab.research.google.com/github/nagamharitha/A-real-time-chat-room/blob/main/Week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Logistic Regression") \
    .getOrCreate()

In [ ]:
pandas_df = pd.read_excel('/content/drive/MyDrive/ML CN7030 /bank (1).xlsx')

In [ ]:
dff = spark.createDataFrame(pandas_df)

In [ ]:
print(dff.count())


In [ ]:
dff.select("deposit").distinct().show()
dff.printSchema()

In [ ]:
print("yes",dff.select("deposit").where("deposit=='yes'").count())
print("No",dff.select("deposit").where("deposit=='no'").count())

In [ ]:
# checking the distinct values in string columns of the dataframe using functional programming
[print("column name",dff[t[0]].name,dff.select(t[0]).distinct().show()) for t in dff.dtypes if t[1]=='string' ]

Display Integer colums


In [ ]:
# taking only integer features
numeric_features = [t[0] for t in dff.dtypes if t[1] == 'int']
dff.select(numeric_features).describe().toPandas().transpose()


In [ ]:
#find out is there any null value in any column
print("total records:",dff.count())
[print(col,"Total null values",dff.where(dff[col].isNull()).count()) for col in dff.columns]

Selecting the features of feature vectors


In [ ]:
# Non Numeric features
df = dff.select('job', 'marital', 'education', 'default',  'housing', 'loan', 'contact', 'duration',  'poutcome', 'deposit')
cols = df.columns
df.printSchema()

In [ ]:
df = dff.select('age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'deposit')
cols = df.columns
df.printSchema()

In [ ]:
# Now we will apply string indexing to categorial variables using pipe line
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
categoricalColumns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    print(stringIndexer.getOutputCol())
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'deposit', outputCol = 'label')
print(label_stringIdx)
stages += [label_stringIdx]
numericCols = ['age','balance', 'duration', 'campaign', 'pdays', 'previous']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
print(assemblerInputs)
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [ ]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
df.show(12,True)

In [ ]:
print(df.select("features").show(1))
print(cols)

In [ ]:
# feature vector is in features  from assembler.
selectedCols = ['label', 'features'] + cols
dfSelected = df.select(selectedCols)
dfSelected.printSchema()

In [ ]:
# this is code for multiple classification using logistic Regression
from pyspark.ml.classification import OneVsRest
from pyspark.ml.classification import LogisticRegression
train, test = dfSelected.randomSplit([0.7, 0.3], seed = 2018)
lr = LogisticRegression(maxIter=100, \

                        featuresCol="features", \

                        labelCol='label')
ovr = OneVsRest(classifier=lr, \
                labelCol='label', \
                featuresCol='features')

ovrModel = ovr.fit(train)
predictionsovr = ovrModel.transform(test)
predictionsovr.show(10)

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Create MulticlassMetrics object
 #Convert predictions and labels to RDD for MulticlassMetrics
prediction_and_labels = predictionsovr.select("prediction", "label") \
    .withColumnRenamed("indexedLabel", "label") \
    .toPandas()  # Convert to Pandas DataFrame for easier manipulation
# Create a confusion matrix using Pandas
confusion_matrix = pd.crosstab(prediction_and_labels['label'], prediction_and_labels['prediction'], rownames=['Actual'], colnames=['Predicted'])

# Plot the confusion matrix using Seaborn and Matplotlib
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

evaluator = MulticlassClassificationEvaluator(\
labelCol="label", predictionCol="prediction",\
metricName="accuracy")
accuracy = evaluator.evaluate(predictionsovr)
print("Test accuracy =  " , accuracy)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
train, test = dfSelected.randomSplit([0.7, 0.3], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
predictions = dtModel.transform(test)

#predictions.select("label","prediction").show(10)
predictions.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)
print("Total Actual Positive",predictions.select("label").where('label == 1.0').count())
print("Total Actual Negative",predictions.select("label").where('label == 0.0').count())
pr = predictions.toPandas()
TruePositive =0
FalsePositive=0
TrueNegative=0
FalseNegative=0
Postive=1.0
Negative=0.0
pos=0
Neg=0

print("Total",len(pr["label"]))
for lbl in range(len(pr["label"])):
  if  pr["prediction"][lbl]==Postive:
    pos+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TruePositive+=1
    else:
      FalsePositive+=1
  if  pr["prediction"][lbl]==Negative:
    Neg+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TrueNegative+=1
    else:
      FalseNegative+=1
print("Total Positive & Negative in Prediction. Pos: ",pos,",Neg",Neg)
print("TruePostive",TruePositive,"FalsePostive",FalsePositive)
print("TrueNegative",TrueNegative,"FalseNegative",FalseNegative)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()

print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

In [ ]:
#RandomForest
from pyspark.ml.classification import RandomForestClassifier
#rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rf = RandomForestClassifier(featuresCol='features', labelCol='label', maxDepth = 4,numTrees=20)
train, test = dfSelected.randomSplit([0.7, 0.3], seed = 2018)
rfModel = rf.fit(train)
predictions = rfModel.transform(test)
pr = predictions.toPandas()
print("Total Actual Positive",predictions.select("label").where('label == 1.0').count())
print("Total Actual Negative",predictions.select("label").where('label == 0.0').count())

evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))
TruePositive =0
FalsePositive=0
TrueNegative=0
FalseNegative=0
Postive=1.0
Negative=0.0
pos=0
Neg=0
print("Total",len(pr["label"]))
for lbl in range(len(pr["label"])):
  if  pr["prediction"][lbl]==Postive:
    pos+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TruePositive+=1
    else:
      FalsePositive+=1
  if  pr["prediction"][lbl]==Negative:
    Neg+=1
    if pr["prediction"][lbl]==pr["label"][lbl]:
      TrueNegative+=1
    else:
      FalseNegative+=1
print("Total positive & Negative in Predictions, Pos",pos,"Neg",Neg)
print("TruePostive",TruePositive,"FalsePostive",FalsePositive)
print("TrueNegative",TrueNegative,"FalseNegative",FalseNegative)

In [ ]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier

ml = GBTClassifier(maxIter=10, featuresCol='features', labelCol='label',maxDepth = 10)
train, test = dfSelected.randomSplit([0.7, 0.2], seed = 2018)
mlModel = ml.fit(train)
predictions = mlModel.transform(test)

evaluator = BinaryClassificationEvaluator()
print("Test Area Under ROC: " + str(evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})))

# Calculate accuracy and F-1 score
#accuracy_evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
#accuracy = accuracy_evaluator.evaluate(predictions.select('label', 'prediction'))

#f1_score_evaluator = MulticlassClassificationEvaluator(metricName='f1')
#f1_score = f1_score_evaluator.evaluate(predictions.select('label', 'prediction'))
#print(accuracy, f1_score)

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
gbt = GBTClassifier(maxIter=10)
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 4, 6])
             .addGrid(gbt.maxBins, [20, 60])
             .addGrid(gbt.maxIter, [10, 20])
             .build())
cv = CrossValidator(estimator=gbt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(train)
predictions = cvModel.transform(test)
evaluator.evaluate(predictions)

0.8999779711421949

In [ ]:
predictions.select('label', 'prediction').show(13)

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  1.0|       0.0|
|  1.0|       1.0|
+-----+----------+
only showing top 13 rows



In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import GBTClassifier, OneVsRest
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
path="/content/drive/MyDrive/ML CN7030 /Week5/bezdekIris.data"
# Start Spark session
spark = SparkSession.builder.appName("GBTClassifier-Multiclass").getOrCreate()

# Load dataset (replace with your path)
# Assuming the order of columns is sepal_length, sepal_width, petal_length, petal_width, species
iris = spark.read.csv(path, header=False, inferSchema=True).toDF("sepal_length", "sepal_width", "petal_length", "petal_width", "species")
# Prepare features
feature_cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
iris = assembler.transform(iris)

# Index labels
indexer = StringIndexer(inputCol="species", outputCol="label")
iris = indexer.fit(iris).transform(iris)

# Train-test split
train, test = iris.randomSplit([0.8, 0.2], seed=42)

# Initialize GBTClassifier
gbt = GBTClassifier(maxIter=20, maxDepth=3, stepSize=0.1)

# Wrap in OneVsRest for multiclass
ovr = OneVsRest(classifier=gbt)

# Train
model = ovr.fit(train)

# Predict
predictions = model.transform(test)

# Evaluate
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy:.4f}")

spark.stop()

Test Accuracy: 0.9583


In [ ]:
iris.printSchema()

root
 |-- 5.1: double (nullable = true)
 |-- 3.5: double (nullable = true)
 |-- 1.4: double (nullable = true)
 |-- 0.2: double (nullable = true)
 |-- Iris-setosa: string (nullable = true)

